# Data Transform

In this notebook, we will ask you a series of questions to evaluate your findings from your EDA. Based on your response & justification, we will ask you to also apply a subsequent data transformation. 

If you state that you will not apply any data transformations for this step, you must **justify** as to why your dataset/machine-learning does not require the mentioned data preprocessing step.

The bonus step is completely optional, but if you provide a sufficient feature engineering step in this project we will add `1000` points to your Kahoot leaderboard score.

You will write out this transformed dataframe as a `.csv` file to your `data/` folder.

**Note**: Again, note that this dataset is quite large. If you find that some data operations take too long to complete on your machine, simply use the `sample()` method to transform a subset of your data.

In [44]:
import pandas as pd
import numpy as np

In [45]:
transactions = pd.read_csv("../data/bank_transactions.csv")
transactions.head()

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,PAYMENT,983.09,C1454812978,36730.24,35747.15,M1491308340,0.00,0.00,0,0
1,PAYMENT,55215.25,C1031766358,99414.00,44198.75,M2102868029,0.00,0.00,0,0
2,CASH_IN,220986.01,C1451868666,7773074.97,7994060.98,C1339195526,924031.48,703045.48,0,0
3,TRANSFER,2357394.75,C458368123,0.00,0.00,C620979654,4202580.45,6559975.19,0,0
4,CASH_OUT,67990.14,C1098978063,0.00,0.00,C142246322,625317.04,693307.19,0,0


## Q1

Does your model contain any missing values or "non-predictive" columns? If so, which adjustments should you take to ensure that your model has good predictive capabilities? Apply your data transformations (if any) in the code-block below.

Answer here

The dataset does not contain duplicates or missing values.

The following features are not predictive and can be dropped:
- `nameOrig`: Origin account name — not useful for prediction.
- `nameDest`: Destination account name — not useful for prediction.
- `isFlaggedFraud`: Output of a hard-coded fraud rule — ineffective and redundant.

In [46]:
df = transactions.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1)
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,983.09,36730.24,35747.15,0.00,0.00,0
1,PAYMENT,55215.25,99414.00,44198.75,0.00,0.00,0
2,CASH_IN,220986.01,7773074.97,7994060.98,924031.48,703045.48,0
3,TRANSFER,2357394.75,0.00,0.00,4202580.45,6559975.19,0
4,CASH_OUT,67990.14,0.00,0.00,625317.04,693307.19,0


## Q2

Do certain transaction types consistently differ in amount or fraud likelihood? If so, how might you transform the type column to make this pattern usable by a machine learning model? Apply your data transformations (if any) in the code-block below.

Answer here

- **TRANSFER** and **CASH_OUT** transactions are the only types associated with fraud.
- These two types also involve significantly higher transaction amounts on average.
- Other types such as **PAYMENT**, **DEBIT**, and **CASH_IN** have much smaller amounts and no fraud cases.

To make this pattern usable for machine learning:
- Apply **one-hot encoding** to represent transaction types as binary columns.

In [47]:
df_encoded = pd.get_dummies(
    data=df,
    columns=['type'],
    dtype='int',
    drop_first=True
)
df_encoded.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,983.09,36730.24,35747.15,0.00,0.00,0,0,0,1,0
1,55215.25,99414.00,44198.75,0.00,0.00,0,0,0,1,0
2,220986.01,7773074.97,7994060.98,924031.48,703045.48,0,0,0,0,0
3,2357394.75,0.00,0.00,4202580.45,6559975.19,0,0,0,0,1
4,67990.14,0.00,0.00,625317.04,693307.19,0,1,0,0,0


## Q3

After exploring your data, you may have noticed that fraudulent transactions are rare compared to non-fraudulent ones. What challenges might this pose when training a machine learning model? What strategies could you use to ensure your model learns meaningful patterns from the minority class? Apply your data transformations (if any) in the code-block below.

Fraudulent transactions make up only 0.13% of the dataset, posing a significant challenge for model training. Most classifiers may default to predicting the majority class (non-fraud) to maximize accuracy, resulting in poor fraud detection.

To address this, the following strategies were considered:

- **Class Weights**: Using `class_weight='balanced'` helps the model give more importance to the minority class without altering the data.
- **SMOTE**: Synthetic Minority Over-sampling Technique generates synthetic examples of the minority class. It should only be applied to the **training set** after splitting.

**Approach Taken**:
Model training began with `class_weight='balanced'` for simplicity and efficiency. If model performance is inadequate, SMOTE will be applied as a follow-up step.

## Bonus (optional)

Are there interaction effects between variables (e.g., fraud and high amount and transaction type) that aren't captured directly in the dataset? Would it be helpful to manually engineer any new features that reflect these interactions? Apply your data transformations (if any) in the code-block below.

Answer Here

Some patterns in the data combine multiple features, like high amounts in certain transaction types. To capture these, new features were created:

- `high_risk_type`: Marks transactions over 400,000 in `TRANSFER` or `CASH_OUT` types, where most fraud happens.

- `orig_diff` and `dest_diff`: Measure balance changes in origin and destination accounts. These don't always match the `amount`, which may show hidden behavior. Including them could help detect irregular or suspicious transactions.

These features help the model see patterns not directly shown in the original data.

In [48]:
df_encoded['high_risk_type'] = (
    ((df_encoded['type_TRANSFER']) | (df_encoded['type_CASH_OUT'])) &
    (df_encoded['amount'] > 400000)
).astype(int)

df_encoded.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,high_risk_type
0,983.09,36730.24,35747.15,0.00,0.00,0,0,0,1,0,0
1,55215.25,99414.00,44198.75,0.00,0.00,0,0,0,1,0,0
2,220986.01,7773074.97,7994060.98,924031.48,703045.48,0,0,0,0,0,0
3,2357394.75,0.00,0.00,4202580.45,6559975.19,0,0,0,0,1,1
4,67990.14,0.00,0.00,625317.04,693307.19,0,1,0,0,0,0


While new features introduces some redundancy with existing variables (`amount` and `type_*`), it captures an important interaction that models might miss. Tree-based models handle such multicollinearity well. Its impact will be evaluated during modeling.

In [49]:
df_encoded.to_csv('../data/bank_transactions_transformed.csv', index=False)